In [1]:
import arcpy
from arcpy.sa import *
import math

arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"C:\Users\ss2596\The University of Waikato\Lars Brabyn - Skye\00 Jake Overton\Njoko_Map\Njoko_Map.gdb"

dem = "aoi_dem_SRTMGL1_Clip"
twi = dem + "_twi"

# Intermediates
filled_dem = "tmp_filled"
flow_dir = "tmp_flow_dir"
flow_acc = "tmp_flow_acc"
slope_rad = "tmp_slope_rad"
tan_slope = "tmp_tan_slope"
adj_acc = "tmp_adj_acc"

arcpy.CheckOutExtension("Spatial")

print("Calculating Topographic Wetness Index for:", dem)

try:
    print("Getting cell size...")
    cell_size = float(arcpy.GetRasterProperties_management(dem, "CELLSIZEX").getOutput(0))

    print("Filling DEM...")
    filled = Fill(dem)
    filled.save(filled_dem)

    print("Flow Direction...")
    fdir = FlowDirection(filled_dem)
    fdir.save(flow_dir)

    print("Flow Accumulation (scaled)...")
    facc = FlowAccumulation(flow_dir) * cell_size
    facc.save(flow_acc)

    print("Slope in radians...")
    slope_deg = Slope(filled_dem, output_measurement="DEGREE")
    slope_rad_raster = slope_deg * (math.pi / 180)
    slope_rad_raster.save(slope_rad)

    print("Calculating tan(slope)...")
    tan_slope_raster = Tan(Raster(slope_rad))
    tan_slope_raster.save(tan_slope)

    print("Adjusting flow accumulation...")
    adjusted_acc = Raster(flow_acc) + 1
    adjusted_acc.save(adj_acc)

    print("Applying Con to avoid divide-by-zero in flat areas...")
    safe_tan = Con(Raster(tan_slope) < 0.001, 0.001, Raster(tan_slope))


    print("Calculating TWI...")
    twi_raster = Ln(Raster(adj_acc) / safe_tan)
    twi_raster.save(twi)

    print("TWI saved as:", twi)

finally:
    print("Cleaning up intermediates...")
    for tmp in [filled_dem, flow_dir, flow_acc, slope_rad, tan_slope, adj_acc]:
        try:
            arcpy.Delete_management(tmp)
            print(f"Deleted: {tmp}")
        except Exception as e:
            print(f"Could not delete {tmp}: {e}")

    for var in [
        "filled", "fdir", "facc", "slope_deg", "slope_rad_raster",
        "tan_slope_raster", "adjusted_acc", "twi_raster"
    ]:
        try:
            del globals()[var]
        except:
            pass

    arcpy.CheckInExtension("Spatial")


Calculating Topographic Wetness Index for: aoi_dem_SRTMGL1_Clip
Getting cell size...
Filling DEM...
Flow Direction...
Flow Accumulation (scaled)...
Slope in radians...
Calculating tan(slope)...
Adjusting flow accumulation...
Applying Con to avoid divide-by-zero in flat areas...
Calculating TWI...
TWI saved as: aoi_dem_SRTMGL1_Clip_twi
Cleaning up intermediates...
Deleted: tmp_filled
Deleted: tmp_flow_dir
Deleted: tmp_flow_acc
Deleted: tmp_slope_rad
Deleted: tmp_tan_slope
Deleted: tmp_adj_acc
